## Описание данных
* TVR Index - Нормированный рейтинг блока (его нужно спрогнозировать). За 1 берется средний рейтинг блока за 2023 год

* Date - Дата выхода блока

* Break flight ID - Идентификатор выхода блока

* Break flight start - Время начала рекламного блока

* Break flight end - Время окончания рекламного блока

* Break content - Содержание блока: Коммерческий (обычная реклама), Спонсорский, Блок анонсов телепередач

* Break distribution - Распространение блока: Network (по всей России), Orbital (по всей России, если не перекрыт локальной рекламой), Local (локальная реклама)

* Programme - Название программы, в которой выходит рекламный блок

* Programme flight start - Время начала программы, в которой выходит рекламный блок

* Programme flight end - Время окончания программы, в которой выходит рекламный блок

* Programme category - Категория программы

* Programme genre - Жанр программы

## Загрузка данных

In [1]:
import pandas as pd
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

In [2]:
# функция для чтения
def read_data(file_path, parse_dates=[]):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    lines = data.split('\n')
    tab_separated_data = StringIO('\n'.join(lines))

    df = pd.read_csv(tab_separated_data, sep='\t', parse_dates=parse_dates, dayfirst=True)

    return df

In [3]:
# чтение
file_path = './data/train.txt'
parse_dates=[1]
df = read_data(file_path, parse_dates)
df

,TVR Index,Date,Break flight ID,Break flight start,Break flight end,Break content,Break distribution,Programme,Programme flight start,Programme flight end,Programme category,Programme genre
0,"0,614692654",2023-01-02,4870830561,8:17:33,8:21:40,Commercial,Network,"Telekanal ""Dobroe utro""",8:00:13,10:00:14,Morning airplay,Entertainment programs
1,"0,869565217",2023-01-02,4870830614,8:34:45,8:38:52,Commercial,Network,"Telekanal ""Dobroe utro""",8:00:13,10:00:14,Morning airplay,Entertainment programs
2,"0,989505247",2023-01-02,4870830629,8:52:19,8:56:23,Commercial,Network,"Telekanal ""Dobroe utro""",8:00:13,10:00:14,Morning airplay,Entertainment programs
3,"0,884557721",2023-01-02,4870830684,8:56:31,8:57:28,Announcement,Network,"Telekanal ""Dobroe utro""",8:00:13,10:00:14,Morning airplay,Entertainment programs
4,"0,83958021",2023-01-02,4870830685,9:12:04,9:16:13,Commercial,Network,"Telekanal ""Dobroe utro""",8:00:13,10:00:14,Morning airplay,Entertainment programs
...,...,...,...,...,...,...,...,...,...,...,...,...
30677,"0,029985007",2023-10-31,5335333196,27:43:40,27:44:07,Announcement,Network,Podkast.Lab,27:27:49,28:12:28,Entertainment talk show,Entertainment programs
30678,"0,029985007",2023-10-31,5335333197,27:44:07,27:48:13,Commercial,Network,Podkast.Lab,27:27:49,28:12:28,Entertainment talk show,Entertainment programs
30679,"0,014992504",2023-10-31,5335333212,28:02:10,28:06:16,Commercial,Network,Podkast.Lab,27:27:49,28:12:28,Entertainment talk show,Entertainment programs
30680,"0,029985007",2023-10-31,5335333228,28:26:20,28:30:25,Commercial,Network,Podkast.Lab,28:12:28,28:56:34,Entertainment talk show,Entertainment programs


In [4]:
# статистика
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30682 entries, 0 to 30681
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   TVR Index               30682 non-null  object        
 1   Date                    30682 non-null  datetime64[ns]
 2   Break flight ID         30682 non-null  int64         
 3   Break flight start      30682 non-null  object        
 4   Break flight end        30682 non-null  object        
 5   Break content           30682 non-null  object        
 6   Break distribution      30682 non-null  object        
 7   Programme               30682 non-null  object        
 8   Programme flight start  30682 non-null  object        
 9   Programme flight end    30682 non-null  object        
 10  Programme category      30682 non-null  object        
 11  Programme genre         30682 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
mem

In [5]:
# статистика 
df.describe()

,Break flight ID
count,3.068200e+04
mean,5.100496e+09
std,1.338851e+08
min,4.870831e+09
25%,4.984457e+09
50%,5.098226e+09
75%,5.215397e+09
max,5.335333e+09


## Обработка данных

In [6]:
import holidays
from sklearn.preprocessing import LabelEncoder
import pickle

Формат даты не удобен для обучения, поэтому необходимо преобразовать. Оптимальным вариантом является номер недели и день недели. Год не несет никакой информации, так как данные даны за один год, а номер недели поможет лучше отслеживать сезонность по неделям и месяцам

In [7]:
# создание атрибута номер недели и день недели
df['Week'] = df['Date'].apply(lambda x: x.isocalendar()[1])
df['Week day'] = df['Date'].apply(lambda x: x.isocalendar()[1])

In [8]:
# создание атрибута праздник
ru_holidays = holidays.RU()
df['holiday'] = df['Date'].apply(lambda x: 1 if x in ru_holidays else 0)

In [9]:
# создание атрибута выходной
df['weekend'] = df['Date'].apply(lambda x: 1 if x.weekday() > 4 or x in ru_holidays else 0)

In [10]:
# удаление атрибута id
df = df.drop('Break flight ID', axis=1)

In [11]:
# удаление атрибута дата
df = df.drop('Date', axis=1)

In [12]:
# функция для преобразования времени
def time_processing(time_str):
    return int(time_str.split(':')[2]) + int(time_str.split(':')[1])*60 + int(time_str.split(':')[0])*3600

In [13]:
# преобразование времени в секунды
df['Break flight start'] = df['Break flight start'].apply(time_processing)
df['Break flight end'] = df['Break flight end'].apply(time_processing)
df['Programme flight start'] = df['Programme flight start'].apply(time_processing)
df['Programme flight end'] = df['Programme flight end'].apply(time_processing)

In [14]:
# создание атрибута длительность рекламы
df['Break duration'] = df['Break flight end'] - df['Break flight start']

In [15]:
# создание атрибута длительность программы
df['Programme duration'] = df['Programme flight end'] - df['Programme flight start']

In [16]:
# преобразование целевого атрибута в число
df['TVR Index'] = df['TVR Index'].apply(lambda x: float(x.replace(',', '.')))

In [17]:
df = df[df['TVR Index'] != 0]

In [18]:
# создание атрибута среднего просмотра для каждой программы
groups = df.groupby(by='Programme')['TVR Index'].mean()
means_dict = {}
for group in groups.index.tolist():
    means_dict[group] = groups[group]

with open('./means_dict.pkl', 'wb') as f:
    pickle.dump(means_dict, f)

In [19]:
df['Programme mean'] = df['Programme'].apply(lambda x: means_dict[x])

In [20]:
# создание словаря для кодировки всех строковых атрибутов
encoder = {}

for column in df.select_dtypes(include="object").columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column].values)
    attr_dict = {}
    for i, val in enumerate(le.classes_.tolist()):
        attr_dict[val] = i
    encoder[column] = attr_dict

In [21]:
# сохранение словаря
with open('./encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [22]:
# вывод обработанных данных
df

,TVR Index,Break flight start,Break flight end,Break content,Break distribution,Programme,Programme flight start,Programme flight end,Programme category,Programme genre,Week,Week day,holiday,weekend,Break duration,Programme duration,Programme mean
0,0.614693,29853,30100,1,1,17,28813,36014,7,1,1,1,1,1,247,7201,1.112456
1,0.869565,30885,31132,1,1,17,28813,36014,7,1,1,1,1,1,247,7201,1.112456
2,0.989505,31939,32183,1,1,17,28813,36014,7,1,1,1,1,1,244,7201,1.112456
3,0.884558,32191,32248,0,1,17,28813,36014,7,1,1,1,1,1,57,7201,1.112456
4,0.839580,33124,33373,1,1,17,28813,36014,7,1,1,1,1,1,249,7201,1.112456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30677,0.029985,99820,99847,0,1,11,98869,101548,2,1,44,44,0,0,27,2679,0.148949
30678,0.029985,99847,100093,1,1,11,98869,101548,2,1,44,44,0,0,246,2679,0.148949
30679,0.014993,100930,101176,1,1,11,98869,101548,2,1,44,44,0,0,246,2679,0.148949
30680,0.029985,102380,102625,1,1,11,101548,104194,2,1,44,44,0,0,245,2646,0.148949


## Обучение модели

In [23]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

In [24]:
X = df.drop('TVR Index', axis=1)
y = df['TVR Index']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

### Случайные лес

In [503]:
forest = RandomForestRegressor(n_jobs=-1, n_estimators=1000)
forest.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [504]:
predict = forest.predict(X_test)
print('mean_absolute_percentage_error:', mean_absolute_percentage_error(y_test, predict))
print('mean_absolute_error:', mean_absolute_error(y_test, predict))
print('mean_squared_error:', mean_squared_error(y_test, predict))

mean_absolute_percentage_error: 0.17477730710087497
mean_absolute_error: 0.12458537577814498
mean_squared_error: 0.03181492994046916


### Catboost

In [28]:
cat = CatBoostRegressor(n_estimators=10000, learning_rate=1e-3)
cat.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [29]:
predict = cat.predict(X_test)
print('mean_absolute_percentage_error:', mean_absolute_percentage_error(y_test, predict))
print('mean_absolute_error:', mean_absolute_error(y_test, predict))
print('mean_squared_error:', mean_squared_error(y_test, predict))

mean_absolute_percentage_error: 0.24838696077801325
mean_absolute_error: 0.16138294490593152
mean_squared_error: 0.04948349202758078


### lightgbm

In [30]:
from lightgbm import LGBMRegressor

In [31]:
lgbm = LGBMRegressor(random_state=123, n_estimators=1000)
lgbm.fit(X_train, y_train)

LGBMRegressor(n_estimators=1000, random_state=123)

In [32]:
predict = lgbm.predict(X_test)
print('mean_absolute_percentage_error:', mean_absolute_percentage_error(y_test, predict))
print('mean_absolute_error:', mean_absolute_error(y_test, predict))
print('mean_squared_error:', mean_squared_error(y_test, predict))

mean_absolute_percentage_error: 0.1782134862235981
mean_absolute_error: 0.12290822340090737
mean_squared_error: 0.030198784472059464


### xgboost

In [26]:
from xgboost import XGBRegressor

In [27]:
xgb = XGBRegressor(objective='reg:gamma', n_estimators=1000, seed=12345, n_jobs=-1)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
             objective='reg:gamma', predictor=None, ...)

In [28]:
predict = xgb.predict(X_test)
print('mean_absolute_percentage_error:', mean_absolute_percentage_error(y_test, predict))
print('mean_absolute_error:', mean_absolute_error(y_test, predict))
print('mean_squared_error:', mean_squared_error(y_test, predict))

mean_absolute_percentage_error: 0.15131776625107266
mean_absolute_error: 0.11519156098348066
mean_squared_error: 0.0281075965898261


In [29]:
with open('./models/xgb.pkl', 'wb') as f:
    pickle.dump(xgb, f)

# Получение предсказаний

In [30]:
import pandas as pd
from io import StringIO
import warnings
import holidays
from sklearn.preprocessing import LabelEncoder
import pickle
warnings.filterwarnings('ignore')

In [31]:
# функция для чтения
def read_data(file_path, parse_dates=[]):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    lines = data.split('\n')
    tab_separated_data = StringIO('\n'.join(lines))

    df = pd.read_csv(tab_separated_data, sep='\t', parse_dates=parse_dates, dayfirst=True)

    return df

In [32]:
# чтение
file_path = './data/test.txt'
parse_dates=[0]
df = read_data(file_path, parse_dates)
df_to_predict = df.copy()

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3329 entries, 0 to 3328
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    3329 non-null   datetime64[ns]
 1   Break flight ID         3329 non-null   int64         
 2   Break flight start      3329 non-null   object        
 3   Break flight end        3329 non-null   object        
 4   Break content           3329 non-null   object        
 5   Break distribution      3329 non-null   object        
 6   Programme               3329 non-null   object        
 7   Programme flight start  3329 non-null   object        
 8   Programme flight end    3329 non-null   object        
 9   Programme category      3329 non-null   object        
 10  Programme genre         3329 non-null   object        
 11  TVR Index Forecast      0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), obje

In [34]:
# создание атрибута номер недели и день недели
df['Week'] = df['Date'].apply(lambda x: x.isocalendar()[1])
df['Week day'] = df['Date'].apply(lambda x: x.isocalendar()[1])

In [35]:
# создание атрибута праздник
ru_holidays = holidays.RU()
df['holiday'] = df['Date'].apply(lambda x: 1 if x in ru_holidays else 0)

In [36]:
# создание атрибута выходной
df['weekend'] = df['Date'].apply(lambda x: 1 if x.weekday() > 4 or x in ru_holidays else 0)

In [37]:
# удаление атрибута id
df = df.drop('Break flight ID', axis=1)

In [38]:
# удаление атрибута дата
df = df.drop('Date', axis=1)

In [39]:
# функция для преобразования времени
def time_processing(time_str):
    return int(time_str.split(':')[2]) + int(time_str.split(':')[1])*60 + int(time_str.split(':')[0])*3600

In [40]:
# преобразование времени в секунды
df['Break flight start'] = df['Break flight start'].apply(time_processing)
df['Break flight end'] = df['Break flight end'].apply(time_processing)
df['Programme flight start'] = df['Programme flight start'].apply(time_processing)
df['Programme flight end'] = df['Programme flight end'].apply(time_processing)

In [41]:
# создание атрибута длительность рекламы
df['Break duration'] = df['Break flight end'] - df['Break flight start']

In [42]:
# создание атрибута длительность программы
df['Programme duration'] = df['Programme flight end'] - df['Programme flight start']

In [43]:
means_dict = pd.read_pickle('./means_dict.pkl')
df['Programme mean'] = df['Programme'].apply(lambda x: means_dict[x])

In [44]:
encoder = pd.read_pickle('./encoder.pkl')

In [45]:
for column in df.select_dtypes(include='object'):
    df[column] = df[column].apply(lambda x: encoder[column][x])

In [46]:
df = df.drop('TVR Index Forecast', axis=1)

In [47]:
xgb = pd.read_pickle('./models/xgb.pkl')

In [48]:
df_to_predict['TVR Index Forecast'] = xgb.predict(df)

In [49]:
df_to_predict.to_excel('./data/predict.xlsx', index=False)

# Атрибуты, используемые для обучения

### Для предсказаниия и обучения были использованы 16 атрибутов

### Атрибуты из исходных данных:
* Break flight start - Время в секундах, во сколько начался рекламный блок
* Break flight end - Время в секундах, во сколько закончился рекламный блок
* Break content - Содержание блока
* Break distribution - Распространение блока
* Programme - Название программы
* Programme flight start - Время в секундах, во сколько началась программа
* Programme flight end - Время в секундах, во сколько закончилась программа
* Programme category - Категория программы
* Programme genre - Жанр программы

### Созданные атрибуты:
* Week - Номер недели
* Week day - День недели
* holiday - Является ли атрибут праздником
* weekend - Является ли атрибут выходным днем
* Break duration - Длительность рекламы в секундах
* Programme duration - Длительность прогаммы в секундах
* Programme mean - Среднее значение целевой переменной для программы

# Алгоритмы обучения

Основные алгоритмы:

* Случайный лес

* CatBoost. Алгоритм на основе градиентного бустинга

* Lightgbm. Алгоритм на основе градиентного бустинга

* XGBoost. Алгоритм на основе градиентного бустинга

# Качество

Лучше всех на обучении и тесте показали себя три алгоритма: 
* Lightgbm (MAPE = 0.178)
* Случайный лес (MAPE = 0.176)
* XGBoost (MAPE = 0.151)

In [52]:
import plotly.express as px
results = {
    "x": ["Random Forest", "Light GBM", "XGBoost", "CatBoost"],
    "y": [0.174, 0.178, 0.151, 0.248]
}

fig = px.bar(results, x="x", y="y", title="MAPE loss (lower - better)")
fig.show()

* Модель случайного леса занимает 1.5 гб, что достаточно много, поэтому в дальнейшем этот алгоритм испоьзоваться не будет
* XGBoost обучается несколько быстрее Lightgbm, но оба эти алгоритма могут быть использованы далее

# Написание функций для API

Ниже мы предлагаем готовый api, который позволяет получить предсказания нашей модели.
Небольшой manual:
* predict_dataset - предсказывает датасет и возвращает датасет с пресказаниями
* predict_one - ghtlcrfpsdftn b djpdhfoftn jlyj pyfxtybt
* read_txt - обрабатывает текстовый файл и возвращает датафрейм
* read_xlsx - читает excel файл и возвращает датафрейм

In [ ]:
import pandas as pd
import holidays

def predict_dataset(df):
    df_to_predict = df.copy()
    # создание атрибута номер недели и день недели
    df['Week'] = df['Date'].apply(lambda x: x.isocalendar()[1])
    df['Week day'] = df['Date'].apply(lambda x: x.isocalendar()[1])
    # создание атрибута праздник
    ru_holidays = holidays.RU()
    df['holiday'] = df['Date'].apply(lambda x: 1 if x in ru_holidays else 0)
    # создание атрибута выходной
    df['weekend'] = df['Date'].apply(lambda x: 1 if x.weekday() > 4 or x in ru_holidays else 0)
    # удаление атрибута id
    df = df.drop('Break flight ID', axis=1)
    # удаление атрибута дата
    df = df.drop('Date', axis=1)
    # функция для преобразования времени
    def time_processing(time_str):
        return int(time_str.split(':')[2]) + int(time_str.split(':')[1])*60 + int(time_str.split(':')[0])*3600
    # преобразование времени в секунды
    df['Break flight start'] = df['Break flight start'].apply(time_processing)
    df['Break flight end'] = df['Break flight end'].apply(time_processing)
    df['Programme flight start'] = df['Programme flight start'].apply(time_processing)
    df['Programme flight end'] = df['Programme flight end'].apply(time_processing)
    # создание атрибута длительность рекламы
    df['Break duration'] = df['Break flight end'] - df['Break flight start']
    # создание атрибута длительность программы
    df['Programme duration'] = df['Programme flight end'] - df['Programme flight start']
    means_dict = pd.read_pickle('./means_dict.pkl')
    df['Programme mean'] = df['Programme'].apply(lambda x: means_dict[x])
    encoder = pd.read_pickle('./encoder.pkl')
    for column in df.select_dtypes(include='object'):
        df[column] = df[column].apply(lambda x: encoder[column][x])
    df = df.drop('TVR Index Forecast', axis=1)
    xgb = pd.read_pickle('./models/xgb.pkl')
    df_to_predict['TVR Index Forecast'] = xgb.predict(df)
    return df_to_predict

def predict_one(df):
    df_to_predict = predict_dataset(df)
    return df_to_predict['TVR Index Forecast'][0]

# функция для чтения txt
def read_txt(file_path, parse_dates=[]):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    lines = data.split('\n')
    tab_separated_data = StringIO('\n'.join(lines))
    df = pd.read_csv(tab_separated_data, sep='\t', parse_dates=parse_dates, dayfirst=True)
    return df

# функция для чтения xlsx
def read_xlsx(file_path):
    df = pd.read_excel(file_path)
    time_arr = ['Break flight start', 'Break flight end','Programme flight start','Programme flight end']
    for column in time_arr:
        df[column] = df[column].apply(lambda x: str(x))
    return df

In [ ]:
# пример вызова

# чтение текста
file_path = './data/test.txt'
parse_dates=[0]
df = read_data(file_path, parse_dates)

# предсказание
pred = predict_dataset(df)

# предсказание 1
pred = predict_one(df.iloc[:1])